In [2]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import CSVLogger, TensorBoard, ReduceLROnPlateau, EarlyStopping
import os
from keras.layers.noise import AlphaDropout
import keras.activations

In [3]:
batch_size = 64
num_classes = 10
epochs = 30
units = 64
experiments = 1
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['hard_sigmoid']
# PReLU is not used, since it does not currently support variable inputs
# optimizers = ['rmsp', 'adam', 'sgd', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam']
optimizers = ['Nadam']
save_dir = os.path.join(os.getcwd(), 'saved_models')


In [4]:
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Train per each activation function

In [ ]:
for act in activations:
    for opt in optimizers:
        print("\nTraining for activation " + act + " with optimizer " + opt)

        # Selecting activation function
        if act == 'sigmoid':
            activation = Activation(keras.activations.sigmoid)
        elif act == 'tanh':
            activation = Activation(keras.activations.tanh)
        elif act == 'relu':
            activation = Activation(keras.activations.relu)
        elif act == 'linear':
            activation = Activation(keras.activations.linear)
        elif act == 'elu':
            activation = Activation(keras.activations.elu)
        elif act == 'softplus':
            activation = Activation(keras.activations.softplus)
        elif act == 'softsign':
            activation = Activation(keras.activations.softsign)
        elif act == 'hard_sigmoid':
            activation = Activation(keras.activations.hard_sigmoid)
        elif act == 'LeakyReLU':
            activation = keras.layers.advanced_activations.LeakyReLU()
        elif act == 'PReLU':
            activation = keras.layers.advanced_activations.PReLU()
        elif act == 'ThresholdedReLU':
            activation = keras.layers.advanced_activations.ThresholdedReLU(theta=0.7)
            

        if opt == 'rmsp':
            optimizer = keras.optimizers.rmsprop()                
        elif opt == 'adam':
            optimizer = keras.optimizers.Adam()
        elif opt == 'sgd':
            optimizer = keras.optimizers.SGD()
        elif opt == 'Adagrad':
            optimizer = keras.optimizers.Adagrad()
        elif opt == 'Adadelta':
            optimizer = keras.optimizers.Adadelta()
        elif opt == 'Adamax':
            optimizer = keras.optimizers.Adamax()
        elif opt == 'Nadam':
            optimizer = keras.optimizers.Nadam()
        
        for i in range(experiments):
            model_name = 'mnist_cnn_' + act + "_" + opt + '_' + str(i + start)
            model = Sequential()
            model.add(Conv2D(32, kernel_size=(3, 3),
                             input_shape=input_shape))
            
            model.add(activation)
            model.add(Conv2D(64, kernel_size=(3, 3)))
            model.add(activation)
            model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
            model.add(Dropout(0.25))
            model.add(Flatten())
            model.add(Dense(128))
            model.add(activation)
            model.add(Dropout(0.5))
            model.add(Dense(num_classes, activation='softmax'))

            model.compile(loss='categorical_crossentropy',
                              optimizer=optimizer,
                              metrics=['accuracy'])
            
            print('-'*30)
            print('Experiment', i)

            csv_logger = CSVLogger('./logs/%s_%d.csv' % (model_name, units), append=False, separator=';')
            reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=2, min_lr=0)
#             tb = TensorBoard(log_dir='./tb_logs/' + model_name + '_' + str(units), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_test, y_test), callbacks=[csv_logger, reduce_lr])

            if not os.path.isdir(save_dir):
                os.makedirs(save_dir)
            model_path = os.path.join(save_dir, model_name + ".h5")
#             model.save(model_path)

            score = model.evaluate(x_test, y_test, verbose=0)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])


Training for activation hard_sigmoid with optimizer Nadam
------------------------------
Experiment 0
Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 286s 5ms/step - loss: 2.3695 - acc: 0.1018 - val_loss: 2.3098 - val_acc: 0.1032
Epoch 2/30
60000/60000 [==============================] - 274s 5ms/step - loss: 2.3125 - acc: 0.1038 - val_loss: 2.3078 - val_acc: 0.0958
Epoch 3/30
60000/60000 [==============================] - 324s 5ms/step - loss: 2.3129 - acc: 0.1016 - val_loss: 2.3027 - val_acc: 0.1135
Epoch 4/30
60000/60000 [==============================] - 339s 6ms/step - loss: 2.3115 - acc: 0.1015 - val_loss: 2.3037 - val_acc: 0.0982
Epoch 5/30
60000/60000 [==============================] - 359s 6ms/step - loss: 2.3124 - acc: 0.1036 - val_loss: 2.3076 - val_acc: 0.1010
Epoch 6/30
60000/60000 [==============================] - 341s 6ms/step - loss: 2.3119 - acc: 0.1026 - val_loss: 2.3069 - val_acc: 0.1032
Epoch 7/30
59968/60

# SeLU
This one has special requirements

In [10]:
for opt in optimizers:
    print("Training for activation SeLU with optimizer " + opt)
    for i in range(experiments):
        model_name = 'mnist_cnn_selu_' + opt + '_' + str(i + start) 
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='selu', kernel_initializer='lecun_normal',
                         input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='selu', kernel_initializer='lecun_normal'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(AlphaDropout(0.2))
        model.add(Flatten())
        model.add(Dense(128, activation='selu', kernel_initializer='lecun_normal'))
        model.add(AlphaDropout(0.2))
        model.add(Dense(num_classes, activation='softmax'))

        if opt == 'rmsp':
            optimizer = keras.optimizers.rmsprop()                
        elif opt == 'adam':
            optimizer = keras.optimizers.Adam()
        elif opt == 'sgd':
            optimizer = keras.optimizers.SGD()
        elif opt == 'Adagrad':
            optimizer = keras.optimizers.Adagrad()
        elif opt == 'Adadelta':
            optimizer = keras.optimizers.Adadelta()
        elif opt == 'Adamax':
            optimizer = keras.optimizers.Adamax()
        elif opt == 'Nadam':
            optimizer = keras.optimizers.Nadam()


        model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        print('-'*30)
        print('Experiment', i+1)

        csv_logger = CSVLogger('./logs/%s_%d.csv' % (model_name, units), append=False, separator=';')
        reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, verbose=0, mode='auto', epsilon=0.0001, cooldown=2, min_lr=0)
#         tb = TensorBoard(log_dir='./tb_logs/' + model_name + '_' + str(units), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger, reduce_lr])

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        model_path = os.path.join(save_dir, model_name + ".h5")
#         model.save(model_path)
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

Training for activation SeLU with optimizer rmsp
------------------------------
Experiment 1
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 291s 5ms/step - loss: 0.2262 - acc: 0.9348 - val_loss: 0.0938 - val_acc: 0.9806
Epoch 2/50
60000/60000 [==============================] - 290s 5ms/step - loss: 0.0951 - acc: 0.9743 - val_loss: 0.1054 - val_acc: 0.9822
Epoch 3/50
60000/60000 [==============================] - 290s 5ms/step - loss: 0.0756 - acc: 0.9797 - val_loss: 0.1003 - val_acc: 0.9850
Epoch 4/50
60000/60000 [==============================] - 290s 5ms/step - loss: 0.0680 - acc: 0.9825 - val_loss: 0.0904 - val_acc: 0.9862
Epoch 5/50
60000/60000 [==============================] - 290s 5ms/step - loss: 0.0634 - acc: 0.9840 - val_loss: 0.1203 - val_acc: 0.9823
Epoch 6/50
60000/60000 [==============================] - 289s 5ms/step - loss: 0.0579 - acc: 0.9862 - val_loss: 0.1092 - val_acc: 0.9859
Epoch 7/50
60000/60000 [=====

Epoch 15/50
60000/60000 [==============================] - 310s 5ms/step - loss: 0.0578 - acc: 0.9823 - val_loss: 0.0594 - val_acc: 0.9855
Epoch 16/50
60000/60000 [==============================] - 315s 5ms/step - loss: 0.0558 - acc: 0.9831 - val_loss: 0.0562 - val_acc: 0.9856
Epoch 17/50
60000/60000 [==============================] - 309s 5ms/step - loss: 0.0526 - acc: 0.9840 - val_loss: 0.0546 - val_acc: 0.9870
Epoch 18/50
60000/60000 [==============================] - 306s 5ms/step - loss: 0.0484 - acc: 0.9849 - val_loss: 0.0602 - val_acc: 0.9862
Epoch 19/50
60000/60000 [==============================] - 315s 5ms/step - loss: 0.0463 - acc: 0.9849 - val_loss: 0.0563 - val_acc: 0.9868
Epoch 20/50
60000/60000 [==============================] - 320s 5ms/step - loss: 0.0456 - acc: 0.9859 - val_loss: 0.0555 - val_acc: 0.9864
Epoch 21/50
60000/60000 [==============================] - 319s 5ms/step - loss: 0.0437 - acc: 0.9862 - val_loss: 0.0513 - val_acc: 0.9873
Epoch 22/50
60000/60000 [==

60000/60000 [==============================] - 320s 5ms/step - loss: 0.0070 - acc: 0.9978 - val_loss: 0.0602 - val_acc: 0.9912
Epoch 30/50
60000/60000 [==============================] - 311s 5ms/step - loss: 0.0069 - acc: 0.9980 - val_loss: 0.0606 - val_acc: 0.9912
Epoch 31/50
60000/60000 [==============================] - 305s 5ms/step - loss: 0.0059 - acc: 0.9980 - val_loss: 0.0609 - val_acc: 0.9912
Epoch 32/50
60000/60000 [==============================] - 306s 5ms/step - loss: 0.0061 - acc: 0.9980 - val_loss: 0.0609 - val_acc: 0.9913
Epoch 33/50
60000/60000 [==============================] - 303s 5ms/step - loss: 0.0060 - acc: 0.9979 - val_loss: 0.0608 - val_acc: 0.9911
Epoch 34/50
60000/60000 [==============================] - 303s 5ms/step - loss: 0.0054 - acc: 0.9983 - val_loss: 0.0606 - val_acc: 0.9911
Epoch 35/50
60000/60000 [==============================] - 303s 5ms/step - loss: 0.0065 - acc: 0.9979 - val_loss: 0.0607 - val_acc: 0.9913
Epoch 36/50
60000/60000 [==============

60000/60000 [==============================] - 301s 5ms/step - loss: 0.0035 - acc: 0.9989 - val_loss: 0.0819 - val_acc: 0.9912
Epoch 44/50
60000/60000 [==============================] - 309s 5ms/step - loss: 0.0035 - acc: 0.9989 - val_loss: 0.0820 - val_acc: 0.9912
Epoch 45/50
60000/60000 [==============================] - 297s 5ms/step - loss: 0.0037 - acc: 0.9989 - val_loss: 0.0821 - val_acc: 0.9912
Epoch 46/50
60000/60000 [==============================] - 296s 5ms/step - loss: 0.0044 - acc: 0.9987 - val_loss: 0.0820 - val_acc: 0.9912
Epoch 47/50
60000/60000 [==============================] - 295s 5ms/step - loss: 0.0033 - acc: 0.9990 - val_loss: 0.0819 - val_acc: 0.9911
Epoch 48/50
60000/60000 [==============================] - 296s 5ms/step - loss: 0.0036 - acc: 0.9989 - val_loss: 0.0822 - val_acc: 0.9912
Epoch 49/50
60000/60000 [==============================] - 295s 5ms/step - loss: 0.0036 - acc: 0.9991 - val_loss: 0.0819 - val_acc: 0.9911
Epoch 50/50
60000/60000 [==============